In [1]:
#PES2UG23CS647
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.5 MB/s eta 0:00:00


In [2]:
#PES2UG23CS647
from dotenv import load_dotenv
load_dotenv()

import getpass
import os

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

if "GOOGLE_API_KEY" not in os.environ:
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

Enter your Google API key: ··········


In [3]:
task = "Write a rejection email to a candidate"

print("--- LAZY PROMPT ---")
print(llm.invoke(task).content)

--- LAZY PROMPT ---
Here's a template for a professional and empathetic rejection email. Remember to customize the bracketed information `[ ]` for each candidate.

---

**Subject: Update on Your Application for [Job Title] at [Company Name]**

Dear [Candidate Name],

Thank you for your interest in [Company Name] and for taking the time to apply for the [Job Title] position. We also appreciate you taking the time to [mention if they had an interview, e.g., "speak with our team," or "complete our assessment"].

We received a significant number of applications from highly qualified individuals, making this a very competitive process. After careful consideration, we have decided to move forward with other candidates whose qualifications and experience were a closer match for the specific requirements of *this particular role* at this time.

This was a difficult decision, and we truly appreciate the time and effort you invested in your application.

We encourage you to keep an eye on our ca

In [4]:
structured_prompt = """
# Context
You are an HR Manager at a quirky startup called 'RocketBoots'.

# Objective
Write a rejection email to a candidate named Bob.

# Constraints
1. Be extremely brief (under 50 words).
2. Do NOT say 'we found someone better'. Say 'the role changed'.
3. Sign off with 'Keep flying'.

# Output Format
Plain text, no subject line.
"""

print("--- STRUCTURED PROMPT ---")
print(llm.invoke(structured_prompt).content)

--- STRUCTURED PROMPT ---
Hi Bob,

Thanks for your interest in RocketBoots. We appreciate you taking the time.

Unfortunately, we won't be moving forward with your application as the role changed significantly during our process.

We wish you the best in your job search.

Keep flying.


In [5]:
#PES2UG23CS647
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)

## ZERO SHOT

In [6]:
#PES2UG23CS647
prompt_zero = "Combine 'Tevin' and 'Tanish' into a new funny word."
print(f"Zero-shot: {llm.invoke(prompt_zero).content}")

Zero-shot: Here are a few funny options combining 'Tevin' and 'Tanish':

1.  **Tevish** (Sounds a bit like "devilish" or a silly creature)
2.  **Vinish** (Like a cleaning product, but with a person's name twist!)
3.  **Tevnish** (A bit clunky and awkward, which can be funny)
4.  **Tashvin** (Sounds a bit like "trash bin" if you say it fast, or a mischievous imp)
5.  **Tanvin** (A bit more straightforward, but still new)

My personal favorite for "funny" is **Vinish** or **Tevish**!


FEW SHOT

In [7]:
prompt_few = """
Combine words into a funny new word. Give a sarcastic definition.

Input: Breakfast + Lunch
Output: Brunch (An excuse to drink alcohol before noon)

Input: Chill + Relax
Output: Chillax (What annoying people say when you are panic attacks)

Input: Angry + Hungry
Output:
"""
print(f"Few-Shot: {llm.invoke(prompt_few).content}")

Few-Shot: Input: Angry + Hungry
Output: Hangry (A legitimate medical condition that grants you permission to be an insufferable monster until someone brings you a sandwich.)


In [8]:
#PES2UG23CS647
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [9]:
#PES2UG23CS647
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# 1. Our Database of Examples
examples = [
    {"input": "The internet is down.", "output": "We are observing connectivity latency."},
    {"input": "This code implies a bug.", "output": "The logic suggests unintended behavior."},
    {"input": "I hate this feature.", "output": "This feature does not align with my preferences."},
]

# 2. Template for ONE example
example_fmt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

# 3. The Few-Shot Container
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_fmt,
    examples=examples
)

# 4. The Final Chain
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Corpo-Speak Translator. Rewrite the input to sound professional."),
    few_shot_prompt,
    ("human", "{text}")
])

chain = final_prompt | llm

print(chain.invoke({"text": "This app sucks."}).content)

The application presents significant opportunities for enhancement.


## 3. Analysis

Using FewShotChatMessagePromptTemplate creates a clear separation between **instructions** and **training examples (data)**.  
This structured format helps the model’s Attention Mechanism focus on:

1. **System Instructions** → Defines the task clearly.
2. **Examples** → Shows the pattern to follow.
3. **User Input** → The actual problem to solve.

Because the model sees examples as structured conversations rather than mixed text, it can better understand context and generate more accurate, consistent responses.

This reduces confusion, improves generalization, and makes prompt design cleaner and reusable.


In [10]:
#PES2UG23CS647
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Example dataset
examples = [
    {"input": "This feature is bad.", "output": "This feature needs improvement."},
    {"input": "The server crashed.", "output": "The system experienced an unexpected shutdown."},
]

example_fmt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_fmt,
    examples=examples
)

In [11]:
# Structured Prompt
structured_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a professional language rewriter."),
    few_shot_prompt,
    ("human", "{text}")
])

chain_structured = structured_prompt | llm

print("WITH Few-Shot Template:")
print(chain_structured.invoke({"text": "This app sucks."}).content)

WITH Few-Shot Template:
The app requires substantial improvements.


In [14]:
# Unstructured Prompt
unstructured_prompt = """
Rewrite professionally:

This feature is bad -> This feature needs improvement.
The server crashed -> The system experienced an unexpected shutdown.

Now rewrite:
This app sucks.
"""

print("\nWITHOUT Few-Shot Template:")
print(llm.invoke(unstructured_prompt).content)


WITHOUT Few-Shot Template:
Here are a few options, depending on the specific aspect that "sucks":

1.  **General/Overall:**
    *   "This application's user experience could be enhanced."
    *   "This application requires significant improvements."
    *   "The functionality of this application is suboptimal."

2.  **If it's about performance:**
    *   "This application exhibits performance challenges."

3.  **If it's about usability:**
    *   "The usability of this application needs attention."

The most common and comprehensive professional rewrite for "This app sucks" is usually: **"This application's user experience could be enhanced."**
